#66

In [1]:
!pip install transformers
!pip install sentencepiece

from transformers import BigBirdTokenizer, BigBirdForMaskedLM
import torch

# 모델과 토크나이저 불러오기
tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-base')
model = BigBirdForMaskedLM.from_pretrained('google/bigbird-roberta-base')
model


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00


Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BigBirdForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BigBirdForMaskedLM(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(50358, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
           

#67

In [2]:
inputs = ["I like reading [MASK].", "I like driving a [MASK].","The world is facing with a [MASK] [MASK] crisis. We are all suffering from infectious diseases."]
answers = ["I like reading book.", "I like driving a car.", "The world is facing with a pandemic crisis. We are all suffering from infectious diseases."]


#68

In [3]:
# 리스트 타입의 빈 컨테이너 변수 준비
encoded_inputs = []
encoded_labels =  []

# inputs와 answers를 zip( )을 통해 묶은 후
# inputs와 answers의 입력 문장 하나씩 각기 i와 l 변수로 불러내어 반복 루프 실행
for i, l in zip(inputs, answers):

  # i와 l변수를 각기 토크나이저로 인코딩 후 컨테이너 변수에 각기 순서대로 저장
  # 특히 아래 두번째 코드는 l변수의 토크나이저 인코딩 후 input_ids만 저장
  encoded_inputs.append(tokenizer(i, return_tensors="pt"))
  encoded_labels.append(tokenizer(l, return_tensors="pt")["input_ids"])


# 69

In [4]:
# encoded_inputs와 encoded_labels를 zip()을 통해 묶고
# 그 둘의 요소를 각기 input과 label이라는 변수로 불러내어 반복 반복 루프 실행
for input, label in zip(encoded_inputs, encoded_labels):

  # 모델에 input과 label 투입
  # input은 **kwargs 형태로서 키(key)와 키값(value) 모두 전달
  outputs = model(**input, labels=label)
  # 손실 추출
  loss = outputs.loss
  # 로짓 추출
  logits = outputs.logits

  # 손실 출력. 이때 item( )은 텐서 loss의 값을 파이썬 숫자로 추출
  print(f"loss：{loss.item()}")

  # len(logits[0])은 로짓의 첫번째 차원의 개수이며
  # range(1, leg(logits[0]))은 1,2,3,...그리고 로짓의 첫번째 차원의 개수까지 범위
  # logits[0][i]는 로짓의 첫번째 차원의 i번째 요소
  # argmax(-1)은 logits[0][i]의 요소의 마지막 축(axis)에서 최대값을 갖는 인덱스를 추출
  # 해당 인덱스를 기준으로 디코딩 후 그 결과를 순서대로 묶어서 출력
  print(f"prediction：{' '.join([tokenizer.decode(logits[0][i].argmax(-1)) for i in range(1, len(logits[0]))])}")

  # lable[0][1:-1]은 lable의 첫번째 차원의 처음과 끝을 제외한 모든 요소를 의미
  # 이를 디코딩 후 출력
  print(f"answer：{tokenizer.decode(label[0][1:-1])}")
  # 출력시 줄바꾸기
  print('\n')


Attention type 'block_sparse' is not possible if sequence_length: 7 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


loss：11.18355655670166
prediction：i like reading it . i
answer：I like reading book.


loss：8.306910514831543
prediction：its like driving a car . a
answer：I like driving a car.


loss：4.29605770111084
prediction：the world is facing with a global health crisis . we are all suffering from infectious diseases . .
answer：The world is facing with a pandemic crisis. We are all suffering from infectious diseases.




# 70

In [5]:
# 런타임 1분 소요

# 코랩이 메모리가 갱신(clear)된 경우만 아래 두 줄 코딩에서
# 주석 기호(#)를 제거하고 실행
#!pip install transformers
#!pip install sentencepiece

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

# 모델명
model_name = 'google/pegasus-xsum'

# 가능한 경우 GPU를 사용하고, 그렇지 않은 경우 CPU를 사용
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 모델 및 토크나이저 불러오기
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)


# 71

In [6]:
# 여러 문장을 한꺼번에 인용할 때는 인용부호로 """를 사용
inputs = [
          """
          Pretraining large neural language models, such as BERT, has led to impressive gains on many natural language processing (NLP) tasks. However, most pretraining efforts focus on general domain corpora, such as newswire and Web. A prevailing assumption is that even domain-specific pretraining can benefit by starting from general-domain language models. Recent work shows that for domains with abundant unlabeled text, such as biomedicine, pretraining language models from scratch results in substantial gains over continual pretraining of general-domain language models.
          """
]

# inputs에 담긴 문장을 토크나이저로 인코딩
batch = tokenizer(inputs, truncation=True, padding='longest', return_tensors="pt").to(device)


#72

In [7]:
# 입력문장의 인코딩 결과가 담긴 batch 변수를 **kwargs 형식으로 투입
# model.generate()가 요약 문장을 생성하고 결과는 인코딩되어 있음
translated = model.generate(**batch)

# 위의 인코딩된 요약 문장을 디코딩
# 디코딩 시 [CLS] 등의 특수 토큰은 무시
generated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

# 디코딩 텐서의 첫번째 요소(?차원 추후 재확인 필요) 출력
print(generated_text[0])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Pretraining large neural language models can lead to substantial gains over continual pretraining of general-domain language models.
